In [1]:
import pickle
from keras.models import load_model
import pandas as pd

In [2]:
pd_data = pd.read_csv('./data/Churn_test.csv')
pd_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,15634602,Hargrave,619,Spain,Male,42,2,0.00,1,1,1,101348.88
1,2,15647311,Hill,608,France,Female,41,1,83807.86,1,0,1,112542.58
2,3,15619304,Onio,502,France,Male,42,8,159660.80,3,1,0,113931.57


In [3]:
pd_data = pd_data.drop(labels=['RowNumber', 'CustomerId', 'Surname'], axis=1)
pd_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,Spain,Male,42,2,0.00,1,1,1,101348.88
1,608,France,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Male,42,8,159660.80,3,1,0,113931.57


In [4]:
with open('./data/label_encoder_gender.pkl', 'rb') as data:
    label_encoder_gender = pickle.load(data)

with open('./data/onehot_encoder_geo.pkl', 'rb') as data:
    onehot_encoder_geo = pickle.load(data)

with open('./data/scalar.pkl', 'rb') as data:
    scalar = pickle.load(data)

In [5]:
pd_data["Gender"] = label_encoder_gender.transform(pd_data["Gender"])
pd_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,Spain,1,42,2,0.00,1,1,1,101348.88
1,608,France,0,41,1,83807.86,1,0,1,112542.58
2,502,France,1,42,8,159660.80,3,1,0,113931.57


In [6]:
geo_array = onehot_encoder_geo.transform(pd_data[["Geography"]]).toarray()
geo_pd = pd.DataFrame(geo_array, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_pd


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0


In [7]:
pd_data =  pd.concat([
    pd_data.drop(labels=['Geography'], axis=1),
    geo_pd
    ],
    axis=1)
pd_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,1,42,2,0.00,1,1,1,101348.88,0.0,0.0,1.0
1,608,0,41,1,83807.86,1,0,1,112542.58,1.0,0.0,0.0
2,502,1,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0


In [8]:
scalar_data_arr = scalar.transform(pd_data)
scalar_data_arr

array([[-0.33880827,  0.91324755,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384, -0.99850112,
        -0.57946723,  1.73494238],
       [-0.45296331, -1.09499335,  0.19986603, -1.38944225,  0.12337621,
        -0.91668767, -1.54035103,  0.97481699,  0.21057501,  1.00150113,
        -0.57946723, -0.57638802],
       [-1.5530028 ,  0.91324755,  0.29493847,  1.03972843,  1.33785683,
         2.53355998,  0.64920267, -1.02583358,  0.23472492,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
model = load_model('./data/model.h5')

In [ ]:
model_prediction_output = model.predict(scalar_data_arr)
print(model_prediction_output)

1/1 [==============================] - 0s 31ms/step
[[0.5806114 ]
 [0.32539454]
 [0.99997705]]


In [17]:
for ind, probability in enumerate(model_prediction_output):
    print(pd_data.iloc[ind])

    print()
    print(probability)
    print("The User might leave the bank" if probability[0] > 0.5 else "User might not leave the bank")
    print()

CreditScore             619.00
Gender                    1.00
Age                      42.00
Tenure                    2.00
Balance                   0.00
NumOfProducts             1.00
HasCrCard                 1.00
IsActiveMember            1.00
EstimatedSalary      101348.88
Geography_France          0.00
Geography_Germany         0.00
Geography_Spain           1.00
Name: 0, dtype: float64

[0.5806114]
The User might leave the bank

CreditScore             608.00
Gender                    0.00
Age                      41.00
Tenure                    1.00
Balance               83807.86
NumOfProducts             1.00
HasCrCard                 0.00
IsActiveMember            1.00
EstimatedSalary      112542.58
Geography_France          1.00
Geography_Germany         0.00
Geography_Spain           0.00
Name: 1, dtype: float64

[0.32539454]
User might not leave the bank

CreditScore             502.00
Gender                    1.00
Age                      42.00
Tenure                    